In [33]:
from nltk import word_tokenize
from collections import defaultdict
import pandas as pd
from collections import Counter
from sklearn.model_selection import train_test_split
import io
import numpy as np
import nltk


In [278]:
df = pd.read_csv('o365Data.csv', dtype=object)

In [279]:
all_subjects = df['EmailSubject'].tolist()

In [280]:
receipt_subjects = [s.lower() for s in all_subjects if isinstance(s,str)]

In [8]:
# import mailbox
# mbox = mailbox.mbox('emails.mbox')
# non_receipt_subjects = []
# for message in mbox:
#     payload = message.get_payload(decode=False)
#     non_receipt_subjects.append(message['Subject'])
# #     if isinstance(payload,list):
# #         non_receipt_emails.append(payload[0])
# #     else:
# #         non_receipt_emails.append(payload) 


In [281]:
gmail_df = pd.read_csv('gmail_training_data.csv', dtype=object)
gmail_df['Subjects'].replace('', np.nan, inplace=True)
gmail_df.dropna(subset=['Subjects'], inplace=True)

x_train = gmail_df['Subjects'].tolist()
y_train = gmail_df['Is_Receipt'].tolist()
y_train = np.array(y_train).astype(np.float).tolist()

x_train = x_train + receipt_subjects
y_train = y_train + np.ones(len(receipt_subjects)).astype(np.float).tolist()

test_df = pd.read_csv('test_data.csv', dtype=object)
test_df['Subjects'].replace('', np.nan, inplace=True)
test_df.dropna(subset=['Subjects'], inplace=True)

x_test = test_df['Subjects'].str.lower().tolist()
y_test = test_df['Is_Receipt'].tolist()
y_test = np.array(y_test).astype(np.float).tolist()


In [32]:
# creates the test set

# import csv

# subs = [s for s in non_receipt_subjects if isinstance(s,str)]
# receipt_words = ['invoice','receipt','amazon','paypal','payment','confirmation','trip','booking','order','reservation','confirmed','itinerary','statement','folio','bill']

# receipt_subs = [s.lower() for s in subs if any(word.lower() in s.lower() for word in receipt_words)]
# non_receipt_subs = [s.lower() for s in subs if not any(word.lower() in s.lower() for word in receipt_words)]

# all_subs = receipt_subs + non_receipt_subs
    
# with open("test_set.csv",'w') as resultFile:
#     wr = csv.writer(resultFile, dialect='excel')
#     for row in receipt_subs:
#         wr.writerows([[row,1]])
#     for row in non_receipt_subs:
#         wr.writerows([[row,0]])

In [237]:
def clean_gmail_subjects(subs): 
    subs = [s for s in subs if isinstance(s,str)]
    
#    remove_words = ['invoice','utf','utf-8','receipt','amazon','paypal','payment','confirmation','hughes','trip','booking','lance','weston','order','reservation','confirmed','itinerary','statement','folio','bill']
    remove_words = ['utf','utf-8']

    return[s.lower() for s in subs if not any(word.lower() in s.lower() for word in remove_words)]

In [10]:
# non_receipt_subjects = clean_gmail_subjects(non_receipt_subjects)

In [11]:

import random

# takes in two lists. the negative list and positive and returns a randomized list off all 
# data along with 1 or 0 labels 
def get_randomized_data_with_labels(zeros_list,ones_list):
    labels = np.zeros(len(zeros_list)).tolist() + np.ones(len(ones_list)).tolist()
    
    all_data = zeros_list + ones_list
    combined = list(zip(all_data, labels))
    random.shuffle(combined)

    all_data[:], labels[:] = zip(*combined)
    return all_data,labels

In [12]:
subjects, labels = get_randomized_data_with_labels(non_receipt_subjects,receipt_subjects)

In [66]:
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
x_test = []
y_test = []
predicted = []

def classify(x_train,y_train,x_test,y_test):
    global predicted
#     x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.33, random_state=42)
    vectorizer = CountVectorizer()
#     clf = MultinomialNB()
    clf = SGDClassifier(loss='hinge', penalty='l1', alpha=1e-4, random_state=42, max_iter=7, tol=None)
    pipe = Pipeline([('vect', vectorizer),
                        ('tfidf', TfidfTransformer()),
                        ('clf', clf)
                        ])
                        
    pipe.fit(x_train, y_train)  
    predicted = pipe.predict(x_test)
    
    return np.mean(predicted == y_test), clf, vectorizer,pipe

In [267]:
for i in range(len(y_train)):
    if not isinstance(y_train[i],float):
        print(str(i) + " " + str(y_train[i]))

26390 1.0
26391 1.0
26392 1.0
26393 1.0
26394 1.0
26395 1.0
26396 1.0
26397 1.0
26398 1.0
26399 1.0
26400 1.0
26401 1.0
26402 1.0
26403 1.0
26404 1.0
26405 1.0
26406 1.0
26407 1.0
26408 1.0
26409 1.0
26410 1.0
26411 1.0
26412 1.0
26413 1.0
26414 1.0
26415 1.0
26416 1.0
26417 1.0
26418 1.0
26419 1.0
26420 1.0
26421 1.0
26422 1.0
26423 1.0
26424 1.0
26425 1.0
26426 1.0
26427 1.0
26428 1.0
26429 1.0
26430 1.0
26431 1.0
26432 1.0
26433 1.0
26434 1.0
26435 1.0
26436 1.0
26437 1.0
26438 1.0
26439 1.0
26440 1.0
26441 1.0
26442 1.0
26443 1.0
26444 1.0
26445 1.0
26446 1.0
26447 1.0
26448 1.0
26449 1.0
26450 1.0
26451 1.0
26452 1.0
26453 1.0
26454 1.0
26455 1.0
26456 1.0
26457 1.0
26458 1.0
26459 1.0
26460 1.0
26461 1.0
26462 1.0
26463 1.0
26464 1.0
26465 1.0
26466 1.0
26467 1.0
26468 1.0
26469 1.0
26470 1.0
26471 1.0
26472 1.0
26473 1.0
26474 1.0
26475 1.0
26476 1.0
26477 1.0
26478 1.0
26479 1.0
26480 1.0
26481 1.0
26482 1.0
26483 1.0
26484 1.0
26485 1.0
26486 1.0
26487 1.0
26488 1.0
26489 1.0


27908 1.0
27909 1.0
27910 1.0
27911 1.0
27912 1.0
27913 1.0
27914 1.0
27915 1.0
27916 1.0
27917 1.0
27918 1.0
27919 1.0
27920 1.0
27921 1.0
27922 1.0
27923 1.0
27924 1.0
27925 1.0
27926 1.0
27927 1.0
27928 1.0
27929 1.0
27930 1.0
27931 1.0
27932 1.0
27933 1.0
27934 1.0
27935 1.0
27936 1.0
27937 1.0
27938 1.0
27939 1.0
27940 1.0
27941 1.0
27942 1.0
27943 1.0
27944 1.0
27945 1.0
27946 1.0
27947 1.0
27948 1.0
27949 1.0
27950 1.0
27951 1.0
27952 1.0
27953 1.0
27954 1.0
27955 1.0
27956 1.0
27957 1.0
27958 1.0
27959 1.0
27960 1.0
27961 1.0
27962 1.0
27963 1.0
27964 1.0
27965 1.0
27966 1.0
27967 1.0
27968 1.0
27969 1.0
27970 1.0
27971 1.0
27972 1.0
27973 1.0
27974 1.0
27975 1.0
27976 1.0
27977 1.0
27978 1.0
27979 1.0
27980 1.0
27981 1.0
27982 1.0
27983 1.0
27984 1.0
27985 1.0
27986 1.0
27987 1.0
27988 1.0
27989 1.0
27990 1.0
27991 1.0
27992 1.0
27993 1.0
27994 1.0
27995 1.0
27996 1.0
27997 1.0
27998 1.0
27999 1.0
28000 1.0
28001 1.0
28002 1.0
28003 1.0
28004 1.0
28005 1.0
28006 1.0
28007 1.0


28908 1.0
28909 1.0
28910 1.0
28911 1.0
28912 1.0
28913 1.0
28914 1.0
28915 1.0
28916 1.0
28917 1.0
28918 1.0
28919 1.0
28920 1.0
28921 1.0
28922 1.0
28923 1.0
28924 1.0
28925 1.0
28926 1.0
28927 1.0
28928 1.0
28929 1.0
28930 1.0
28931 1.0
28932 1.0
28933 1.0
28934 1.0
28935 1.0
28936 1.0
28937 1.0
28938 1.0
28939 1.0
28940 1.0
28941 1.0
28942 1.0
28943 1.0
28944 1.0
28945 1.0
28946 1.0
28947 1.0
28948 1.0
28949 1.0
28950 1.0
28951 1.0
28952 1.0
28953 1.0
28954 1.0
28955 1.0
28956 1.0
28957 1.0
28958 1.0
28959 1.0
28960 1.0
28961 1.0
28962 1.0
28963 1.0
28964 1.0
28965 1.0
28966 1.0
28967 1.0
28968 1.0
28969 1.0
28970 1.0
28971 1.0
28972 1.0
28973 1.0
28974 1.0
28975 1.0
28976 1.0
28977 1.0
28978 1.0
28979 1.0
28980 1.0
28981 1.0
28982 1.0
28983 1.0
28984 1.0
28985 1.0
28986 1.0
28987 1.0
28988 1.0
28989 1.0
28990 1.0
28991 1.0
28992 1.0
28993 1.0
28994 1.0
28995 1.0
28996 1.0
28997 1.0
28998 1.0
28999 1.0
29000 1.0
29001 1.0
29002 1.0
29003 1.0
29004 1.0
29005 1.0
29006 1.0
29007 1.0


29907 1.0
29908 1.0
29909 1.0
29910 1.0
29911 1.0
29912 1.0
29913 1.0
29914 1.0
29915 1.0
29916 1.0
29917 1.0
29918 1.0
29919 1.0
29920 1.0
29921 1.0
29922 1.0
29923 1.0
29924 1.0
29925 1.0
29926 1.0
29927 1.0
29928 1.0
29929 1.0
29930 1.0
29931 1.0
29932 1.0
29933 1.0
29934 1.0
29935 1.0
29936 1.0
29937 1.0
29938 1.0
29939 1.0
29940 1.0
29941 1.0
29942 1.0
29943 1.0
29944 1.0
29945 1.0
29946 1.0
29947 1.0
29948 1.0
29949 1.0
29950 1.0
29951 1.0
29952 1.0
29953 1.0
29954 1.0
29955 1.0
29956 1.0
29957 1.0
29958 1.0
29959 1.0
29960 1.0
29961 1.0
29962 1.0
29963 1.0
29964 1.0
29965 1.0
29966 1.0
29967 1.0
29968 1.0
29969 1.0
29970 1.0
29971 1.0
29972 1.0
29973 1.0
29974 1.0
29975 1.0
29976 1.0
29977 1.0
29978 1.0
29979 1.0
29980 1.0
29981 1.0
29982 1.0
29983 1.0
29984 1.0
29985 1.0
29986 1.0
29987 1.0
29988 1.0
29989 1.0
29990 1.0
29991 1.0
29992 1.0
29993 1.0
29994 1.0
29995 1.0
29996 1.0
29997 1.0
29998 1.0
29999 1.0
30000 1.0
30001 1.0
30002 1.0
30003 1.0
30004 1.0
30005 1.0
30006 1.0


30907 1.0
30908 1.0
30909 1.0
30910 1.0
30911 1.0
30912 1.0
30913 1.0
30914 1.0
30915 1.0
30916 1.0
30917 1.0
30918 1.0
30919 1.0
30920 1.0
30921 1.0
30922 1.0
30923 1.0
30924 1.0
30925 1.0
30926 1.0
30927 1.0
30928 1.0
30929 1.0
30930 1.0
30931 1.0
30932 1.0
30933 1.0
30934 1.0
30935 1.0
30936 1.0
30937 1.0
30938 1.0
30939 1.0
30940 1.0
30941 1.0
30942 1.0
30943 1.0
30944 1.0
30945 1.0
30946 1.0
30947 1.0
30948 1.0
30949 1.0
30950 1.0
30951 1.0
30952 1.0
30953 1.0
30954 1.0
30955 1.0
30956 1.0
30957 1.0
30958 1.0
30959 1.0
30960 1.0
30961 1.0
30962 1.0
30963 1.0
30964 1.0
30965 1.0
30966 1.0
30967 1.0
30968 1.0
30969 1.0
30970 1.0
30971 1.0
30972 1.0
30973 1.0
30974 1.0
30975 1.0
30976 1.0
30977 1.0
30978 1.0
30979 1.0
30980 1.0
30981 1.0
30982 1.0
30983 1.0
30984 1.0
30985 1.0
30986 1.0
30987 1.0
30988 1.0
30989 1.0
30990 1.0
30991 1.0
30992 1.0
30993 1.0
30994 1.0
30995 1.0
30996 1.0
30997 1.0
30998 1.0
30999 1.0
31000 1.0
31001 1.0
31002 1.0
31003 1.0
31004 1.0
31005 1.0
31006 1.0


31906 1.0
31907 1.0
31908 1.0
31909 1.0
31910 1.0
31911 1.0
31912 1.0
31913 1.0
31914 1.0
31915 1.0
31916 1.0
31917 1.0
31918 1.0
31919 1.0
31920 1.0
31921 1.0
31922 1.0
31923 1.0
31924 1.0
31925 1.0
31926 1.0
31927 1.0
31928 1.0
31929 1.0
31930 1.0
31931 1.0
31932 1.0
31933 1.0
31934 1.0
31935 1.0
31936 1.0
31937 1.0
31938 1.0
31939 1.0
31940 1.0
31941 1.0
31942 1.0
31943 1.0
31944 1.0
31945 1.0
31946 1.0
31947 1.0
31948 1.0
31949 1.0
31950 1.0
31951 1.0
31952 1.0
31953 1.0
31954 1.0
31955 1.0
31956 1.0
31957 1.0
31958 1.0
31959 1.0
31960 1.0
31961 1.0
31962 1.0
31963 1.0
31964 1.0
31965 1.0
31966 1.0
31967 1.0
31968 1.0
31969 1.0
31970 1.0
31971 1.0
31972 1.0
31973 1.0
31974 1.0
31975 1.0
31976 1.0
31977 1.0
31978 1.0
31979 1.0
31980 1.0
31981 1.0
31982 1.0
31983 1.0
31984 1.0
31985 1.0
31986 1.0
31987 1.0
31988 1.0
31989 1.0
31990 1.0
31991 1.0
31992 1.0
31993 1.0
31994 1.0
31995 1.0
31996 1.0
31997 1.0
31998 1.0
31999 1.0
32000 1.0
32001 1.0
32002 1.0
32003 1.0
32004 1.0
32005 1.0


32906 1.0
32907 1.0
32908 1.0
32909 1.0
32910 1.0
32911 1.0
32912 1.0
32913 1.0
32914 1.0
32915 1.0
32916 1.0
32917 1.0
32918 1.0
32919 1.0
32920 1.0
32921 1.0
32922 1.0
32923 1.0
32924 1.0
32925 1.0
32926 1.0
32927 1.0
32928 1.0
32929 1.0
32930 1.0
32931 1.0
32932 1.0
32933 1.0
32934 1.0
32935 1.0
32936 1.0
32937 1.0
32938 1.0
32939 1.0
32940 1.0
32941 1.0
32942 1.0
32943 1.0
32944 1.0
32945 1.0
32946 1.0
32947 1.0
32948 1.0
32949 1.0
32950 1.0
32951 1.0
32952 1.0
32953 1.0
32954 1.0
32955 1.0
32956 1.0
32957 1.0
32958 1.0
32959 1.0
32960 1.0
32961 1.0
32962 1.0
32963 1.0
32964 1.0
32965 1.0
32966 1.0
32967 1.0
32968 1.0
32969 1.0
32970 1.0
32971 1.0
32972 1.0
32973 1.0
32974 1.0
32975 1.0
32976 1.0
32977 1.0
32978 1.0
32979 1.0
32980 1.0
32981 1.0
32982 1.0
32983 1.0
32984 1.0
32985 1.0
32986 1.0
32987 1.0
32988 1.0
32989 1.0
32990 1.0
32991 1.0
32992 1.0
32993 1.0
32994 1.0
32995 1.0
32996 1.0
32997 1.0
32998 1.0
32999 1.0
33000 1.0
33001 1.0
33002 1.0
33003 1.0
33004 1.0
33005 1.0


33905 1.0
33906 1.0
33907 1.0
33908 1.0
33909 1.0
33910 1.0
33911 1.0
33912 1.0
33913 1.0
33914 1.0
33915 1.0
33916 1.0
33917 1.0
33918 1.0
33919 1.0
33920 1.0
33921 1.0
33922 1.0
33923 1.0
33924 1.0
33925 1.0
33926 1.0
33927 1.0
33928 1.0
33929 1.0
33930 1.0
33931 1.0
33932 1.0
33933 1.0
33934 1.0
33935 1.0
33936 1.0
33937 1.0
33938 1.0
33939 1.0
33940 1.0
33941 1.0
33942 1.0
33943 1.0
33944 1.0
33945 1.0
33946 1.0
33947 1.0
33948 1.0
33949 1.0
33950 1.0
33951 1.0
33952 1.0
33953 1.0
33954 1.0
33955 1.0
33956 1.0
33957 1.0
33958 1.0
33959 1.0
33960 1.0
33961 1.0
33962 1.0
33963 1.0
33964 1.0
33965 1.0
33966 1.0
33967 1.0
33968 1.0
33969 1.0
33970 1.0
33971 1.0
33972 1.0
33973 1.0
33974 1.0
33975 1.0
33976 1.0
33977 1.0
33978 1.0
33979 1.0
33980 1.0
33981 1.0
33982 1.0
33983 1.0
33984 1.0
33985 1.0
33986 1.0
33987 1.0
33988 1.0
33989 1.0
33990 1.0
33991 1.0
33992 1.0
33993 1.0
33994 1.0
33995 1.0
33996 1.0
33997 1.0
33998 1.0
33999 1.0
34000 1.0
34001 1.0
34002 1.0
34003 1.0
34004 1.0


34905 1.0
34906 1.0
34907 1.0
34908 1.0
34909 1.0
34910 1.0
34911 1.0
34912 1.0
34913 1.0
34914 1.0
34915 1.0
34916 1.0
34917 1.0
34918 1.0
34919 1.0
34920 1.0
34921 1.0
34922 1.0
34923 1.0
34924 1.0
34925 1.0
34926 1.0
34927 1.0
34928 1.0
34929 1.0
34930 1.0
34931 1.0
34932 1.0
34933 1.0
34934 1.0
34935 1.0
34936 1.0
34937 1.0
34938 1.0
34939 1.0
34940 1.0
34941 1.0
34942 1.0
34943 1.0
34944 1.0
34945 1.0
34946 1.0
34947 1.0
34948 1.0
34949 1.0
34950 1.0
34951 1.0
34952 1.0
34953 1.0
34954 1.0
34955 1.0
34956 1.0
34957 1.0
34958 1.0
34959 1.0
34960 1.0
34961 1.0
34962 1.0
34963 1.0
34964 1.0
34965 1.0
34966 1.0
34967 1.0
34968 1.0
34969 1.0
34970 1.0
34971 1.0
34972 1.0
34973 1.0
34974 1.0
34975 1.0
34976 1.0
34977 1.0
34978 1.0
34979 1.0
34980 1.0
34981 1.0
34982 1.0
34983 1.0
34984 1.0
34985 1.0
34986 1.0
34987 1.0
34988 1.0
34989 1.0
34990 1.0
34991 1.0
34992 1.0
34993 1.0
34994 1.0
34995 1.0
34996 1.0
34997 1.0
34998 1.0
34999 1.0
35000 1.0
35001 1.0
35002 1.0
35003 1.0
35004 1.0


In [282]:

accuracy, clf, vectorizer,pipe = classify(x_train,y_train,x_test,y_test)
print(accuracy)


0.9077966101694915


In [274]:
from sklearn import metrics
print(metrics.classification_report(y_test, predicted))

             precision    recall  f1-score   support

        0.0       0.89      0.91      0.90       645
        1.0       0.93      0.91      0.92       830

avg / total       0.91      0.91      0.91      1475



In [284]:
z = [x_test[i] for i in range(len(x_test)) if y_test[i] == 0 and predicted[i] == 1]
z


['your reservation confirmation for the melting pot - bellevue',
 'confirmation gtrev/',
 '10 packing mistakes that can ruin your trip',
 'road trip: honey coffee at alma matter + cosmonaut (tacoma) on\r\n sunday, august 26',
 'your seated confirmation for dining at the melting pot - bellevue',
 'top 10 road trip chains',
 'interested in medical billing school?',
 'udacity: billing address update required',
 'greece: athens & santorini 6-night trip from seattle, $899',
 '36% of millennials take second jobs to save for a down payment',
 'road trip upgrade!',
 'japan: 7-night trip with seattle flights, now $1299',
 'you have authorized a payment to fandango media llc',
 'reservation confirmation at luna on saturday, june 16',
 'one question about your recent trip',
 'let mylo find homeowners insurance that could lower your mortgage\r\n payment!',
 'save $10 with alexa on your first order',
 'your reservation confirmation for the melting pot - bellevue',
 'fwd: reservation reminder - may 

In [290]:
body = "your card was declined"

pipe.predict([body])


array([0.])

In [238]:
def is_receipt(subject):
    return pipe.predict([subject])[0] == 1

In [240]:
def show_most_informative_features(vectorizer, clf, n=20):
    feature_names = vectorizer.get_feature_names()
    coefs_with_fns = sorted(zip(clf.coef_[0], feature_names))
    top = zip(coefs_with_fns[:n], coefs_with_fns[:-(n + 1):-1])
    for (coef_1, fn_1), (coef_2, fn_2) in top:
        print ("\t%.4f\t%-15s\t\t%.4f\t%-15s" % (coef_1, fn_1, coef_2, fn_2))

In [291]:
show_most_informative_features(vectorizer,clf,1000)

	-3.5522	failure        		14.2754	receipt        
	-3.3977	hughes         		10.7942	order          
	-2.8682	rhughes509     		10.2278	invoice        
	-2.2763	so             		9.6772	reservation    
	-2.0499	off            		7.8141	trip           
	-1.8712	rate           		7.4636	itinerary      
	-1.7419	decompression  		6.9881	confirmation   
	-1.6727	cubs           		6.8980	payment        
	-1.6626	touch          		6.7016	ride           
	-1.5498	declined       		5.2514	ref            
	-1.5099	update         		5.2475	booking        
	-1.4018	in             		5.0728	folio          
	-1.2849	financial      		4.9115	bill           
	-1.2677	meetup         		4.6182	billing        
	-1.2512	history        		4.5615	statement      
	-1.1435	associates     		4.3584	national       
	-1.1348	ifttt          		4.1787	enterprise     
	-1.1187	news           		3.8346	de             
	-1.1019	seattle        		3.4842	uber           
	-1.0613	game           		3.4029	stay           
	-1.0517	tefoq0ug

In [20]:
import eli5
eli5.show_weights(clf, top=10)

Weight?,Feature
+13.847,x10400
+13.105,x9369
+10.556,x4215
+10.071,x7139
+9.616,x10597
+8.421,x9619
+8.016,x12546
+6.830,x10696
+6.144,x7218
+6.037,x3318
